# Q-LoRA

### 한국어 QA 시스템 최적화

##### 0. 환경 설정

In [ ]:
!pip install peft datasets transformers hf_transfer bitsandbytes

In [ ]:
# !python -m pip install --upgrade pip

In [ ]:
# !pip install typing_extensions==4.7.1 --upgrade

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, PeftModel
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

##### 1. 모델 로드

In [ ]:
model_name = 'NCSOFT/Llama-VARCO-8B-Instruct'

In [ ]:
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# 토크나이저 및 모델 로드

tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

##### 2. 데이터 준비

In [ ]:
# 데이터셋 로드
dataset = load_dataset('KorQuAD/squad_kor_v1')

In [ ]:
dataset['train'][0]

In [ ]:
def preprocess_data(examples):
    
    inputs = ["질문: " + q + "\n문맥: " + c for q, c in zip(examples["question"], examples["context"])]
    answer_texts = [a["text"][0] if len(a["text"]) > 0 else "" for a in examples["answers"]]


    model_inputs = tokenizer(
        inputs,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    labels = tokenizer(
        answer_texts,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )['input_ids']
    
    max_length = model_inputs['input_ids'].shape[1]
    labels = labels[:, :max_length]


    labels[labels == tokenizer.pad_token_id] = -100

    model_inputs['labels'] = labels

    return model_inputs

In [ ]:
# 데이터셋 전처리 적용
train_dataset = dataset['train'].map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names) 
val_dataset = dataset['validation'].map(preprocess_data, batched=True, remove_columns=dataset['validation'].column_names) 

In [ ]:
train_dataset[0]

In [ ]:
print(tokenizer.decode(train_dataset[0]['input_ids'], skip_special_tokens=True))

##### 3. 모델 학습 준비

In [ ]:
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
    target_modules=['q_proj', 'v_proj'],
    task_type='CAUSAL_LM'
)

In [ ]:
model = get_peft_model(base_model, lora_config)

model.enable_input_require_grads()
model.gradient_checkpointing_enable()
model.print_trainable_parameters()

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
training_args = TrainingArguments(
    output_dir='./q_lora_korqa',
    save_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    weight_decay=0.01,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
    report_to='none'
)


In [ ]:
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=val_dataset,
tokenizer=tokenizer,
data_collator=data_collator
)

##### 4. 모델 학습 (Fine-tuning)

In [ ]:
trainer.train()

##### 5. 학습된 모델 활용

(1) 모델 추론

In [ ]:
from transformers import AutoConfig

trained_model_path = './q_lora_korqa/checkpoint-1888'

config = AutoConfig.from_pretrained(model_name)
config.save_pretrained(trained_model_path)

In [ ]:
from peft import PeftModel
from transformers import pipeline

base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype='auto', device_map='auto')
model = PeftModel.from_pretrained(base_model, trained_model_path)

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
question = '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'
context = '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

input_text = f"질문: {question}\n문맥: {context}"

output = qa_pipeline(input_text, max_new_tokens=50, temperature=0.2, top_p=0.8)

print(output)

(2) HuggingFace Hub 

In [ ]:
model.save_pretrained('./q_lora_korqa')
tokenizer.save_pretrained('./q_lora_korqa')

model.push_to_hub('codespace01/q_lora_korqa')
tokenizer.push_to_hub('codespace01/q_lora_korqa')

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    "NCSOFT/Llama-VARCO-8B-Instruct",
    torch_dtype='auto',
    device_map='auto'
)

model = PeftModel.from_pretrained(
    base_model,
    'codespace01/q_lora_korqa'
)


tokenizer = AutoTokenizer.from_pretrained('codespace01/q_lora_korqa')

In [ ]:
inputs = tokenizer('바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?', return_tensors='pt').to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))